In [1]:
def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)
    ax.tick_params(width=0.5)

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def loadRippleFrames(expt):
    try:
        with open(expt.LFPFilePath() + '/' + 'ripple_frames.pkl', 'rb') as f:
            rFrames = pickle.load(f)
    except:
        r_path = [fn for fn in locate('*keep_ripples.npy', expt.LFPFilePath(), ignore=None)][0]
        rtimes = np.load(r_path)
        ts = np.arange(expt.num_frames()) * expt.frame_period()
        ripple_frames = []
        for t in xrange(len(rtimes)):
            ripple_frames.append(find_nearest(ts, rtimes[t][0]))
        ripple_frames = np.asarray(ripple_frames)
        rFrames = ripple_frames[ripple_frames < expt.num_frames()]
        with open(expt.LFPFilePath() + '/' + 'ripple_frames.pkl', 'wb') as f:
            pickle.dump(rFrames, f)
    return rFrames

def ROI_planeID(expt, label):
    signals = expt.imaging_dataset().signals()[label]['rois']
    nROIs = len(signals)
    planeID = []
    for i in xrange(nROIs):
        planeID.append(signals[i]['polygons'][0][0][2])

    return np.asarray(planeID).astype('int')

def resample_trace(fluorescence_trace, number_of_data_points):
    y = fluorescence_trace
    x = np.arange(0, len(y))
    f = interpolate.interp1d(x,y)
    xnew = np.linspace(x.min(), x.max(), num = number_of_data_points)
    ynew = f(xnew)
    return ynew

def get_immobility_frames(expt, immobility_cutoff = 0.2):
    return np.where(expt.velocity()[0] < immobility_cutoff)[0]

def butter_bandpass_filter(signal, lowcut, highcut, Fs = 20000, order = 4):
    nyq_f = 0.5 * Fs
    low = lowcut / nyq_f
    high = highcut / nyq_f
    b,a = butter(order, [low, high], btype = 'band')
    return lfilter(b,a,signal)